In [ ]:
#| default_exp core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from collections import Counter
from itertools import cycle

import numpy as np
from fastcore.basics import patch

from matatena.exceptions import ColumnFullError

# Basic rules

Knucklebones is a simple game in which a dice is rolled and one of the two players must place it in their corresponding board. Each player can only choose the column in which the dice is placed, and if any dice is already placed in that column, the following is put behind. If there are more than one dice with the same number, their score is added and then multiplied by the number of times it appears. For example, if a column has two 6s, their values add up to 24, but if a column has a 6 and a 5, the total score would be 11.

> The basic board is a 3x3 grid.

# Definition of the board

Each game board (one for each player) can be represented, for example, with a `np.array`. But we may probably benefit from having a "global" object that controls both of the players to keep track of both boards and the score in a more simple manner.

In [ ]:
#| export

class Game():
    """
    Class that controls the whole game. It keeps track of both boards.
    """
    def __init__(self,
                 n_players=2, # Number of players.
                 board_size=3, # Size of the board. It is a squared grid of `board_size`x`board_size`
                 ):
        self.n_players = n_players
        self.board_size = board_size
        self.boards = np.zeros(shape=(n_players, board_size, board_size))
        self.current_player = self.choose_initial_player()
        self._players = cycle(range(n_players))

    def __repr__(self):
        """
        Representation of the game state.
        The current player is marked with an *.
        """
        player0 = "Player 1 *" if self.current_player==0 else "Player 1"
        player1= "Player 2 *" if self.current_player==1 else "Player 2"
        
        return "\n".join([player0,
                          str(self.boards[0]),
                          player1,
                          str(self.boards[1])])
        
    def choose_initial_player(self):
        """
        The initial player is chosen at random at the beggining of the game.
        """
        return np.random.choice([0,1])
    
    def _change_player(self):
        """
        Changes the current player.
        """
        # return next(self._players)
        self.current_player = next(self._players)

    def is_done(self):
        """
        The game is considered finished when one of the boards is completed.
        This can be checked by checking if there are any 0s left in a board.
        """
        return (not (self.boards[0] == 0).any()) or (not (self.boards[1] == 0).any())

    def add_dice(self,
                 player, # Player to add the dice to.
                 column, # Column where we want to place the dice.
                 dice, # Dice to place.
                 ):
        """
        Adds a dice to the corresponding player in the specified column
        and removes same dice from the opposite board if they are in the same column.
        """
        board_column = self.boards[player][:,column]
        idxs_dice = np.where(board_column==0)[0]
        if len(idxs_dice)==0: raise ColumnFullError # Can't place dice in a full column.
        board_column[idxs_dice[0]] = dice

        ## Remove dice from the opponent's board
        opposite_players_mask = np.arange(self.boards.shape[0]) != player
        opposite_columns = self.boards[opposite_players_mask][:,column]
        opposite_columns = self.boards[opposite_players_mask].squeeze()[:,column]
        idxs = np.where(opposite_columns==dice)[0]
        self.boards[opposite_players_mask,idxs,column] = 0
        
        return self.is_done()

In [ ]:
matatena = Game()
assert not matatena.is_done()
matatena

Player 1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

In [ ]:
matatena = Game()
matatena.boards[1] = np.ones_like(matatena.boards[1])
assert matatena.is_done()
matatena

Player 1 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

In [ ]:
matatena = Game()
matatena.add_dice(player=0, column=0, dice=6)
print(matatena)
matatena.add_dice(player=0, column=0, dice=6)
print(matatena)

Player 1
[[6. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 1
[[6. 0. 0.]
 [6. 0. 0.]
 [0. 0. 0.]]
Player 2 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [ ]:
matatena = Game()
matatena.add_dice(player=0, column=0, dice=6)
matatena.add_dice(player=0, column=0, dice=6)
matatena.add_dice(player=0, column=0, dice=6)
# matatena.add_dice(player=0, column=0, dice=6)

False

In [ ]:
matatena = Game()
matatena.current_player = 0
matatena.boards[1] = np.ones_like(matatena.boards[1])
print(matatena)
matatena.add_dice(player=0, column=0, dice=1)
assert not (matatena.boards[1][:,0] == 1).any()
matatena

Player 1 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


Player 1 *
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2
[[0. 1. 1.]
 [0. 1. 1.]
 [0. 1. 1.]]

## Calculating the score

In [ ]:
#| export

@patch
def score(self: Game,
          player, # Number of the player we want to calculate the score.
          ):
    """
    Returns the calculated score for a player. 
    If there are numbers repeated in a column, 
    their values must be added and multiplied by the number of repetitions. 
    Otherwise, they are added. If there are repreated and non-repeated in the same column, 
    the repeated are summed and multiplied by the number of repetitions and then the result is added to the non-repeated.
    """
    total = 0
    for col in self.boards[player].T: # Transposed to iterate over the columns
        cntr = Counter(col)
        sum_col = sum([n*reps*reps for n, reps in cntr.items()])
        total += sum_col
    return total

In [ ]:
matatena = Game()
matatena.boards[0] = np.array([[1,0,0],
                               [1,2,3],
                               [4,2,5]])
assert matatena.score(0) == 8+8+8

Now that we know how to calculate the score, we can `patch` the `__repr__` method of our class to show the score of each player as well:

In [ ]:
#| export

@patch
def __repr__(self: Game):
    """
    Representation of the game state.
    The current player is marked with an *.
    """
    player0 = f"Player 1 ({self.score(0)})"
    player1 = f"Player 2 ({self.score(1)})"
    player0 = f"{player0} *" if self.current_player==0 else player0
    player1= f"{player1} *" if self.current_player==1 else player1
    
    return "\n".join([player0,
                      str(self.boards[0]),
                      player1,
                      str(self.boards[1])])

In [ ]:
matatena = Game()
assert not matatena.is_done()
matatena

Player 1 (0.0)
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2 (0.0) *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

In [ ]:
#| export

@patch
def _pad_player_board(self: Game,
                      player: str, # Player __repr__
                      board: str, # Board __repr__
                      ):
    """
    Pads the representation of a player indicator and a board
    so that each line has the same characters.
    """
    max_len = max(max(map(len, board.split("\n"))), len(player))
    return "\n".join(map(lambda x: x.ljust(max_len, " "), "\n".join([player, board]).split("\n")))

In [ ]:
#| export

@patch
def _join_players_boards(self: Game,
                         playerboard1: str, # Padded playerboard obtained from `._pad_player_board()`.
                         playerboard2: str, # Padded playerboard obtained from `._pad_player_board()`.
                         separator: str = " | ", # Character used as a separator.
                         ):
    """
    Join two padded player boards with a separator in between.
    """
    n_lines = max(playerboard1.count("\n"), playerboard2.count("\n")) + 1 # Last line doesn't have \n.
    separator = "\n".join([separator]*n_lines)

    split_lines = zip(playerboard1.split("\n"), separator.split("\n"))
    res = "\n".join([s1 + s2 for s1, s2 in split_lines])
    split_lines = zip(res.split("\n"), playerboard2.split("\n"))
    res = "\n".join([s1 + s2 for s1, s2 in split_lines])
    return res

In [ ]:
#| export

@patch
def __repr__(self: Game):
    """
    Representation of the game state.
    The current player is marked with an *.
    """
    player0 = f"Player 1 ({self.score(0)})"
    player1 = f"Player 2 ({self.score(1)})"
    player0 = f"{player0} *" if self.current_player==0 else player0
    player1= f"{player1} *" if self.current_player==1 else player1

    board0 = str(self.boards[0])
    board1 = str(self.boards[1])
    
    padded0 = self._pad_player_board(player0, board0)
    padded1 = self._pad_player_board(player1, board1)
    
    return self._join_players_boards(padded0, padded1)

In [ ]:
matatena = Game()
matatena

Player 1 (0.0) * | Player 2 (0.0)
[[0. 0. 0.]      | [[0. 0. 0.]   
 [0. 0. 0.]      |  [0. 0. 0.]   
 [0. 0. 0.]]     |  [0. 0. 0.]]  

## Full turn

A full turn in a Matatena game must follow the following steps:

- Check the current player
- Draw a random dice
- Place the dice in a column
    - Check if the column is not full -> The dice can be placed (If it can't be placed the game is ended)
- Change the current player to the next one in the list
- Repeat

In [ ]:
#| export

@patch
def play_turn(self: Game):
    """
    Plays a full turn.
    """
    dice = np.random.choice(range(1,7))
    print(f"Dice to place: {dice}")
    print(self)
    column = int(input("Select column: "))
    self.add_dice(player=self.current_player,
                  column=column,
                  dice=dice)
    self._change_player()

In [ ]:
#| notest
matatena = Game()
print(matatena.current_player)
matatena.play_turn()
# The player is asked for an input and the rolled dice is placed
# in the chosen column.
print(matatena.current_player)
matatena

0
Dice to place: 5
Player 1 (0.0) * | Player 2 (0.0)
[[0. 0. 0.]      | [[0. 0. 0.]   
 [0. 0. 0.]      |  [0. 0. 0.]   
 [0. 0. 0.]]     |  [0. 0. 0.]]  
0


Player 1 (5.0) * | Player 2 (0.0)
[[5. 0. 0.]      | [[0. 0. 0.]   
 [0. 0. 0.]      |  [0. 0. 0.]   
 [0. 0. 0.]]     |  [0. 0. 0.]]  